### Lab 6

In [1]:
import sys
import random
import numpy as np
from statistics import mode
from scipy.stats import norm
from future.utils import iteritems
from sklearn.cluster import KMeans
from sklearn import metrics, datasets
from sklearn.preprocessing import normalize
from scipy.stats import multivariate_normal as mvn
from sklearn.model_selection import train_test_split

1. Implement the EM-algorithm to find a Gaussian NBC for the digits dataset from SciKitLearn (you can of course also use the MNIST_Light set from Lab 5, but for initial tests the digits data set is more convenient, since it is smaller in various aspects). You may assume (conditional) independence between the attributes, i.e., the covariances can be assumed to be simply the variances over each attribute. Split the data set in 70% training and 30% test data.

SciKitLearn digits

In [2]:
X,y = datasets.load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=42)
# Normalize the pixelvalues from range(0,17) to range(0,1)
X_train = X_train/16
X_test = X_test/16

Extract max and min values of mean and variance which is used later to know in what range the initialized variance and mean in the EM algorithm should span between.

In [3]:
m = np.zeros(64)
var = np.zeros(64)
for i in range(64):
    m[i] = np.mean(X_train[i])
    var[i] = np.var(X_train[i])
print('mean min:',min(m),'mean max:',max(m))
print('var min:',min(var),'var max:',max(var))

mean min: 0.224609375 mean max: 0.3955078125
var min: 0.09405517578125 var max: 0.1777191162109375


EM-algorithm

In [4]:
class EM:
    
    def __init__(self, num_classes):
        self.num_classes = num_classes

        
    def fit(self, X_train, eps): #, keep=True
        
        # Uniform distribution for mean, variance and priors
        self.gaussian = {cl: {'mean':np.random.uniform(1,size=X_train[0].size), #len(X_train[0])
                               'var':np.random.uniform(0.5,size=X_train[0].size)} for cl in range(self.num_classes)}
        self.priors = {k:1/self.num_classes for k in range(self.num_classes)}
        
        for i in range(100): # while not diff > epsilon approximately 0.01
                            # diff = abs(norm(mean_old)-norm(mean))
                            # mean_old = mean
            # E_step
            r = np.zeros((len(X_train), self.num_classes))
            
            for index,x in enumerate(X_train):
                num = np.prod([norm.pdf(x, self.gaussian[k]['mean'], np.sqrt(self.gaussian[k]['var']))
                      for k in range(self.num_classes)], axis = 1)
                num = [self.priors[k]*num[k] for k in range(self.num_classes)]
                den = np.sum(num)
                r[index,:] = num/den
                
            # M_step
            N, c = r.shape  
            self.priors = {c:np.sum(r[:,c])/N for c in range(self.num_classes)}
            mean = np.zeros(len(X_train[0]))
            for c in range(self.num_classes):
                mean = np.sum([r[index,c]*xi for index, xi in enumerate(X_train)], axis=0)/np.sum(r[:,c])
                self.gaussian[c]['mean'] = mean
                self.gaussian[c]['var'] = np.sum([r[index,c]*np.diag(np.outer(xi, xi)) for index, xi in enumerate(X_train)], axis=0)/np.sum(r[:,c]) - np.diag(np.outer(mean, mean)) + eps

       # return self.gaussian, self.priors
            
    def predict(self, X_test):
        # Builds the P matrix of size (number images, number of classes)
        row, col = X_test.shape
        G = len(self.gaussian)
        P = np.zeros((row, G))
        for c, g in iteritems(self.gaussian):
            mean, var = g['mean'], g['var']
            # We can choose the digit class using c = argmax_c(logP(x|c)+logP(c))
            # mvn.logpdf() since we wants log of the probability density function
            
            P[:,c] = mvn.logpdf(X_test, mean=mean, cov=var, allow_singular=True) + np.log(self.priors[c])

        # axis = 1 since we want argmax along the rows    
        return np.argmax(P, axis=1)
    

In [5]:
em = EM(10)
eps = 1e-2
em.fit(X_train, eps)
y_pred_em = em.predict(X_train)

2. Use the result of the EM-algorithm (the found distribution parameters) to cluster the training data (essentially, using the resulting classifier to do a prediction over the training data). Produce a confusion matrix over the known labels for the training data and your EM-generated clusters. What do you see?

*Homogeneity: score between 0.0 and 1.0. 1.0 stands for perfectly homogeneous labeling*

*Completeness: score between 0.0 and 1.0. 1.0 stands for perfectly complete labeling*

*V_measure: harmonic mean of the first two*

In [6]:
print('(Homogenity, Completeness, V-measure)')
metrics.homogeneity_completeness_v_measure(y_train, y_pred_em, beta=1.0)

(Homogenity, Completeness, V-measure)


(0.68083051421843, 0.6934526680732898, 0.6870836268303576)

3. If necessary, find a way to "repair" the cluster assignments so that you can do a prediction run over the test data, from which you can compare the results with your earlier implementation of the Gaussian NBC.

In [15]:
def repair(X_train, y_train, y_pred):
    """Repair by studying closest centroid."""
    
    k_classes = list(set(y_train))
    c_true = {}
    c_pred = {}
    for kcl in k_classes:
        c_true[kcl] = np.mean([X for i, X in enumerate(X_train) if y_train[i] == kcl], axis=0)
        c_pred[kcl] = np.mean([X for i, X in enumerate(X_train) if y_pred[i] == kcl], axis=0)

    pred2true = {}
    for kcl in k_classes:
        pred2true[kcl] = np.argmin([np.linalg.norm(c_pred[kcl] - c_true_v) for c_true_v in c_true.values()])
    
    y_new = []
    for y in y_pred:
        y_new.append(pred2true[y])
    return np.asarray(y_new)


def repair_ind(X_train, y_train, y_pred):
    """Repair by studying true labels compared to predicted and the search for the max occurance 
    to assign each label to its corresponding prediction."""
    
    k_classes = list(set(y_train))
    pred2true = {}
    for kcl in k_classes:
        indices = [i for i, x in enumerate(X_train) if y_pred[i] == kcl]
        unique, counts= np.unique(y_train[indices], return_counts=True)
        pred2true[kcl] = unique[np.argmax(counts)]
        
    y_new = []
    for y in y_pred:
        y_new.append(pred2true[y])
    return np.asarray(y_new)

*repair_ind()*

Classification report and confusion matrix

In [16]:
y_pred_repair_ind = repair_ind(X_train, y_train, y_pred_em)

print("Classification report SKLearn:\n%s\n"
  % (metrics.classification_report(y_train, y_pred_repair_ind)))
print("Confusion matrix SKLearn:\n%s" % metrics.confusion_matrix(y_train, y_pred_repair_ind))

Classification report SKLearn:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       125
           1       0.47      0.46      0.47       132
           2       0.84      0.87      0.85       130
           3       0.44      0.74      0.55       129
           4       0.48      0.43      0.45       121
           5       0.96      0.73      0.83       116
           6       0.98      0.97      0.98       128
           7       0.69      0.98      0.81       124
           8       0.70      0.80      0.74       131
           9       0.00      0.00      0.00       121

    accuracy                           0.70      1257
   macro avg       0.65      0.70      0.67      1257
weighted avg       0.66      0.70      0.67      1257


Confusion matrix SKLearn:
[[123   2   0   0   0   0   0   0   0   0]
 [  0  61  16   0  44   1   2   1   7   0]
 [  0   0 113   2   6   0   0   1   8   0]
 [  0   0   1  96   0   1   0   6  25   0]
 [  0  49  

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Homogenity, Completeness, V-measure

In [17]:
print('(Homogenity, Completeness, V-measure)')
metrics.homogeneity_completeness_v_measure(y_train, y_pred_repair_ind, beta=1.0)

(Homogenity, Completeness, V-measure)


(0.6709536090053164, 0.7131908565248847, 0.6914277967533776)

*repair()*

Classification report and confusion matrix

In [18]:
y_pred_repair = repair(X_train, y_train, y_pred_em)

print("Classification report SKLearn:\n%s\n"
  % (metrics.classification_report(y_train, y_pred_repair)))
print("Confusion matrix SKLearn:\n%s" % metrics.confusion_matrix(y_train, y_pred_repair))

Classification report SKLearn:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       125
           1       0.47      0.46      0.47       132
           2       0.84      0.87      0.85       130
           3       0.00      0.00      0.00       129
           4       0.48      0.43      0.45       121
           5       0.96      0.73      0.83       116
           6       0.98      0.97      0.98       128
           7       0.69      0.98      0.81       124
           8       0.70      0.80      0.74       131
           9       0.43      0.78      0.55       121

    accuracy                           0.70      1257
   macro avg       0.65      0.70      0.67      1257
weighted avg       0.65      0.70      0.67      1257


Confusion matrix SKLearn:
[[123   2   0   0   0   0   0   0   0   0]
 [  0  61  16   0  44   1   2   1   7   0]
 [  0   0 113   0   6   0   0   1   8   2]
 [  0   0   1   0   0   1   0   6  25  96]
 [  0  49  

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Homogenity, Completeness, V-measure

In [11]:
print('(Homogenity, Completeness, V-measure)')
metrics.homogeneity_completeness_v_measure(y_train, y_pred_repair, beta=1.0)

(Homogenity, Completeness, V-measure)


(0.6709536090053164, 0.7131908565248846, 0.6914277967533775)

4. Use now also the k-Means implementation from SciKitLearn and compare the results to yours (they should be similar at least in the sense that there are classes that are more clearly separated from the rest than others for both approaches). 

In [12]:
kmeans = KMeans(n_clusters=10, random_state=42).fit(X_train)
km_pred = kmeans.predict(X_train)
km_pred_repair = repair_ind(X_train, y_train, km_pred)

Classification report and confusion matrix

In [13]:
print("Classification report SKLearn:\n%s\n"
  % (metrics.classification_report(y_train, km_pred_repair)))
print("Confusion matrix SKLearn:\n%s" % metrics.confusion_matrix(y_train, km_pred_repair))

Classification report SKLearn:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       125
           1       0.73      0.86      0.79       132
           2       0.85      0.82      0.83       130
           3       0.46      0.88      0.60       129
           4       0.99      0.89      0.94       121
           5       0.92      0.85      0.88       116
           6       0.97      0.97      0.97       128
           7       0.84      0.90      0.87       124
           8       0.79      0.75      0.77       131
           9       0.00      0.00      0.00       121

    accuracy                           0.79      1257
   macro avg       0.75      0.79      0.77      1257
weighted avg       0.75      0.79      0.77      1257


Confusion matrix SKLearn:
[[124   0   0   0   1   0   0   0   0   0]
 [  0 114  14   0   0   1   3   0   0   0]
 [  1   4 106  11   0   0   0   2   6   0]
 [  0   0   1 114   0   2   0   6   6   0]
 [  0   4  

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Homogenity, Completeness, V-measure

In [14]:
print('(Homogenity, Completeness, V-measure)')
metrics.homogeneity_completeness_v_measure(y_train, km_pred_repair, beta=1.0)

(Homogenity, Completeness, V-measure)


(0.7291640278659127, 0.7769060897534751, 0.7522783528480621)